In [1]:
# importing libraries
import numpy as np
import cv2
import os # dealing with directories
import math
from tqdm import tqdm
from math import pi
import tensorflow as tf

TRAIN_DIR_WHAT = 'C:/Users/gabri/Documents/Uni/Year 2/Engineering Design Project/Hard drive/Raw Data/DATABASE/What'
TRAIN_DIR_HUNGRY = 'C:/Users/gabri/Documents/Uni/Year 2/Engineering Design Project/Hard drive/Raw Data/DATABASE/Hungry'
TRAIN_DIR_ANGRY = 'C:/Users/gabri/Documents/Uni/Year 2/Engineering Design Project/Hard drive/Raw Data/DATABASE/Angry'
TRAIN_DIR_WHAT_ROTATED = 'C:/Users/gabri/Documents/Uni/Year 2/Engineering Design Project/Hard drive/Raw Data/DATABASE/What_rotated'
TRAIN_DIR_HUNGRY_ROTATED = 'C:/Users/gabri/Documents/Uni/Year 2/Engineering Design Project/Hard drive/Raw Data/DATABASE/Hungry_rotated'
TRAIN_DIR_ANGRY_ROTATED = 'C:/Users/gabri/Documents/Uni/Year 2/Engineering Design Project/Hard drive/Raw Data/DATABASE/Angry_rotated'
IMG_W_O = 96
IMG_W_F = 54
IMG_H = 54
IMG_CROP = 21
IMG_CROP_1 = IMG_CROP - 1
IMG_CROP_2 = IMG_W_O - IMG_CROP - 1
TIME_FRAMES = 10 

In [2]:
def readVideo(videoFile):
    cap = cv2.VideoCapture(videoFile)
    vid = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            img = cv2.cvtColor(np.array(frame), cv2.COLOR_BGR2GRAY)
            frame = cv2.resize(img, (IMG_W_O, IMG_H))
            img = frame[0:IMG_H, IMG_CROP_1:IMG_CROP_2]
            vid.append(np.array(img, dtype = np.float32))
        else: 
            break
    cap.release()
    return vid 

In [3]:
def readVideoRotated(videoFile):
    cap = cv2.VideoCapture(videoFile)
    vid = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            img = cv2.cvtColor(np.array(frame), cv2.COLOR_BGR2GRAY)
            frame = cv2.resize(img, (IMG_W_F, IMG_H))
            vid.append(np.array(img, dtype = np.float32))
        else: 
            break
    cap.release()
    return vid 

In [4]:
def sample(vid):
    sample_width = math.ceil(len(vid) / TIME_FRAMES)
    new_vid = []
    i = 0
    while (i < len(vid)):
        new_vid.append(np.array(vid[i]))
        i = i + sample_width
    while (len(new_vid) > TIME_FRAMES):
        new_vid = new_vid[:-1]
    while (len(new_vid) < TIME_FRAMES):
        new_vid.append(new_vid[-1])
    new_vid = np.array(new_vid)
    return new_vid

In [5]:
def flipImages(vid):
    X_flip = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (IMG_H, IMG_W_F, 1))
    tf_img = tf.image.flip_left_right(X)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in vid:
            img = np.array(img).reshape(54, 54, 1)
            flipped_imgs = sess.run([tf_img], feed_dict = {X:img})
            X_flip.extend(flipped_imgs)
    X_flip = np.array(X_flip, dtype = np.float32)
    return X_flip

In [6]:
def shade(vid, K):
    shaded_vid = []
    for img in vid:
        img = np.array(img, dtype = np.float)
        img = img*K
        img[img > 255] = 255
        shaded_vid.append(img)
        vid_shaded = np.array(vid_shaded, dtype = np.float32)
    return shaded_vid

In [7]:
def add_salt_pepper_noise(vid):
    # Need to produce a copy as to not modify the original image
    vid_copy = vid.copy()
    row, col, _ = vid_copy[0].shape
    salt_vs_pepper = 0.2
    amount = 0.004
    num_salt = np.ceil(amount * vid_copy[0].size * salt_vs_pepper)
    num_pepper = np.ceil(amount * vid_copy[0].size * (1.0 - salt_vs_pepper))
    
    for X_img in vid_copy:
        # Add Salt noise
        coords = [np.random.randint(0, i - 1, int(num_salt)) for i in X_img.shape]
        X_img[coords[0], coords[1], :] = 1

        # Add Pepper noise
        coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in X_img.shape]
        X_img[coords[0], coords[1], :] = 0
    return vid_copy

In [26]:
def rotateImages(vid, angle):
    vid_rotated = []
    tf.reset_default_graph()
    X = tf.placeholder(tf.float32, shape = (IMG_H, IMG_W_F, 1))
    radian = angle * pi / 180 
    tf_img = tf.contrib.image.rotate(X, radian)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for img in vid:
            img = np.array(img).reshape(54, 54, 1)
            rotated_img = sess.run([tf_img], feed_dict = {X:img})
            vid_rotated.extend(rotated_img)

    vid_rotated = np.array(vid_rotated, dtype = np.float32)
    return vid_rotated

In [9]:
def createTrainingData():
    trainingData = []
    for video in tqdm(os.listdir(TRAIN_DIR_WHAT)):
        videoPath = os.path.join(TRAIN_DIR_WHAT, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_short.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_short), [1, 0, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_HUNGRY)):
        videoPath = os.path.join(TRAIN_DIR_HUNGRY, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_short.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_short), [0, 1, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_ANGRY)):
        videoPath = os.path.join(TRAIN_DIR_ANGRY, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_short.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_short), [0, 0, 1]])
    np.save('trainingData.npy', trainingData) # save the training data in a numpy file
    return trainingData

In [10]:
train_data = createTrainingData()

100%|██████████| 471/471 [13:33<00:00,  1.69s/it]


In [ ]:
def createRotatedTrainingData():
    trainingData = []
    for video in tqdm(os.listdir(TRAIN_DIR_WHAT_ROTATED)):
        videoPath = os.path.join(TRAIN_DIR_WHAT_ROTATED, video) # full path of image
        vid = readVideoRotated(videoPath)
        vid_short = sample(vid)
        trainingData.append([np.array(vid_short), [1, 0, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_HUNGRY_ROTATED)):
        videoPath = os.path.join(TRAIN_DIR_HUNGRY_ROTATED, video) # full path of image
        vid = readVideoRotated(videoPath)
        vid_short = sample(vid)
        trainingData.append([np.array(vid_short), [0, 1, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_ANGRY_ROTATED)):
        videoPath = os.path.join(TRAIN_DIR_ANGRY_ROTATED, video) # full path of image
        vid = readVideoRotated(videoPath)
        vid_short = sample(vid)
        trainingData.append([np.array(vid_short), [0, 0, 1]])
    np.save('trainingDataRotated.npy', trainingData) # save the training data in a numpy file
    return trainingData

In [ ]:
train_data_rotated = createRotatedTrainingData()

In [11]:
def createFlippedTrainingData():
    trainingData = []
    for video in tqdm(os.listdir(TRAIN_DIR_WHAT)):
        videoPath = os.path.join(TRAIN_DIR_WHAT, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_flipped = flipImages(vid_short)
        vid_flipped.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_flipped), [1, 0, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_HUNGRY)):
        videoPath = os.path.join(TRAIN_DIR_HUNGRY, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_flipped = flipImages(vid_short)
        vid_flipped.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_flipped), [0, 1, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_ANGRY)):
        videoPath = os.path.join(TRAIN_DIR_ANGRY, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_flipped = flipImages(vid_short)
        vid_flipped.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_flipped), [0, 0, 1]])
    np.save('trainingDataFlipped.npy', trainingData) # save the training data in a numpy file
    return trainingData

In [12]:
train_data_flipped = createFlippedTrainingData()

100%|██████████| 471/471 [13:26<00:00,  1.61s/it]


In [13]:
def createShadedTrainingData():
    trainingData = []
    for video in tqdm(os.listdir(TRAIN_DIR_WHAT)):
        videoPath = os.path.join(TRAIN_DIR_WHAT, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_shaded = shade(vid_short, 0.9)
        vid_lighter = shade(vid_short, 1.1)
        vid_shaded.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        vid_lighter.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_shaded), [1, 0, 0]])
        trainingData.append([np.array(vid_lighter), [1, 0, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_HUNGRY)):
        videoPath = os.path.join(TRAIN_DIR_HUNGRY, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_shaded = shade(vid_short, 0.9)
        vid_lighter = shade(vid_short, 1.1)
        vid_shaded.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        vid_lighter.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_shaded), [0, 1, 0]])
        trainingData.append([np.array(vid_lighter), [0, 1, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_ANGRY)):
        videoPath = os.path.join(TRAIN_DIR_ANGRY, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_shaded = shade(vid_short, 0.9))
        vid_lighter = shade(vid_short, 1.1))
        vid_shaded.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        vid_lighter.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_shaded), [0, 0, 1]])
        trainingData.append([np.array(vid_lighter), [0, 0, 1]])
    np.save('trainingDataShaded.npy', trainingData) # save the training data in a numpy file
    return trainingData

In [14]:
train_data_shaded = createShadedTrainingData()

100%|██████████| 471/471 [13:20<00:00,  1.59s/it]


In [27]:
def createNewRotatedTrainingData():
    trainingData = []
    for video in tqdm(os.listdir(TRAIN_DIR_WHAT)):
        videoPath = os.path.join(TRAIN_DIR_WHAT, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_rotated1 = rotateImages(vid_short, 15)
        vid_rotated2 = rotateImages(vid_short, 345)
        vid_rotated1.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        vid_rotated2.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_rotated1), [1, 0, 0]])
        trainingData.append([np.array(vid_rotated2), [1, 0, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_HUNGRY)):
        videoPath = os.path.join(TRAIN_DIR_HUNGRY, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_rotated1 = rotateImages(vid_short, 15)
        vid_rotated2 = rotateImages(vid_short, 345)
        vid_rotated1.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        vid_rotated2.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_rotated1), [0, 1, 0]])
        trainingData.append([np.array(vid_rotated2), [0, 1, 0]])
    for video in tqdm(os.listdir(TRAIN_DIR_ANGRY)):
        videoPath = os.path.join(TRAIN_DIR_ANGRY, video) # full path of image
        vid = readVideo(videoPath)
        vid_short = sample(vid)
        vid_rotated1 = rotateImages(vid_short, 15)
        vid_rotated2 = rotateImages(vid_short, 345)
        vid_rotated1.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        vid_rotated2.reshape(-1, TIME_FRAMES, IMG_H, IMG_W_F, 1)
        trainingData.append([np.array(vid_rotated1), [0, 0, 1]])
        trainingData.append([np.array(vid_rotated2), [0, 0, 1]])
    np.save('trainingDataRotatedNew.npy', trainingData) # save the training data in a numpy file
    return trainingData

In [28]:
new_rotated_training_data = createNewRotatedTrainingData()


 33%|███▎      | 165/503 [04:11<07:59,  1.42s/it]


 65%|██████▌   | 328/503 [09:28<06:43,  2.30s/it]


 98%|█████████▊| 491/503 [14:52<00:17,  1.43s/it]


100%|██████████| 503/503 [15:11<00:00,  1.65s/it]

 33%|███▎      | 153/468 [04:32<10:19,  1.97s/it]


 68%|██████▊   | 316/468 [09:58<04:13,  1.67s/it]


100%|██████████| 468/468 [14:02<00:00,  1.87s/it]

  2%|▏         | 11/471 [00:20<14:48,  1.93s/it]


 37%|███▋      | 176/471 [05:33<08:00,  1.63s/it]


 72%|███████▏  | 339/471 [11:06<03:49,  1.74s/it]


100%|██████████| 471/471 [14:53<00:00,  1.89s/it]
